Importing Modules

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np

# Importing data
gh_database = pd.read_csv("co2_emissions_kt_by_country.csv")
countries = gh_database['country_name'].unique().tolist()
p = dict(zip(gh_database['country_name'], gh_database['country_code']))



/home/shravan/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Using linear regression to be able to predict the values
X = gh_database[['country_name', 'year']]
y = gh_database['value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

encoder = OrdinalEncoder()
X_train_encoded = X_train.copy()
X_train_encoded['country_name'] = encoder.fit_transform(X_train[['country_name']])
X_test_encoded = X_test.copy()
X_test_encoded['country_name'] = encoder.transform(X_test[['country_name']])

model = LinearRegression()
model.fit(X_train_encoded, y_train)


0.0101375201812276


In [4]:
# Function to take in inputs & send out output to the gradio interface
def predict_emissions(country, year):
    country_encoded = encoder.transform([[country]])
    data = [[country_encoded[0][0], year]]
    prediction = model.predict(data)
    country_data = gh_database[gh_database['country_name'] == country]
    emissions_data = country_data['value']
    years = country_data['year']
    fig, ax = plt.subplots()
    ax.plot(years, emissions_data)
    ax.set_xlabel("Year")
    ax.set_ylabel("Emissions")
    ax.set_title(f"CO2 Emissions for {country}")
    ax.set_xticklabels(years,rotation=90)
    fig.canvas.draw()
    plot_img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    plot_img = plot_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.clf()
    return ('{:.2f} metric tons per capita'.format(prediction[0]), plot_img)

# Specify the input components (Specified under List)
inputs = [
    gr.inputs.Dropdown(countries, label='Country'),
    gr.inputs.Slider(minimum=1990, maximum=2030, step=1, default=1999, label='Year')
]

# Specify the output components (Can be specified under a list)
output = [
    gr.outputs.Textbox(label='CO2 Emissions'),
    gr.outputs.Image(type='numpy')
]

# Defining what our webapp must consist
interface = gr.Interface(fn=predict_emissions, inputs=inputs, outputs=output, title='CO2 Emissions Predictor')
# To deploy the webapp directly to the web browser or notebook
interface.launch() # Use variable argument 'share=True' for creating a public URL


/home/shravan/.local/lib/python3.11/site-packages/gradio/inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/shravan/.local/lib/python3.11/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/shravan/.local/lib/python3.11/site-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/shravan/.local/lib/python3.11/site-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/home/shravan/.local/lib/python3.11/site-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and wil

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/home/shravan/.local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/home/shravan/.local/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/tmp/ipykernel_270/1779538766.py:13: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(years,rotation=90)
